# Cross-validation

With the code developed so far, it is possible to train an ANN and provide an estimate of the results it would offer in its real execution (with unseen patterns, represented by a test set). However, in this last aspect there are two factors to consider, as a consequence of the non-deterministic nature of the process we are following:

- The partitioning of the set of patterns into training/test is random (hold out), and is therefore overly dependent on good or bad luck in choosing training and test patterns.
- ANN training is not deterministic, as the initialisation of the weights is random. As before, it is too dependent on good or bad luck to start the training at a good or bad starting point.

For these two reasons, the test result of a single training is not significant when assessing the goodness of fit of the model in the presence of unseen patterns. To solve this problem, the experiment is repeated several times and the results are averaged. This can be implemented in a simple way by means of a loop; however, it is necessary to do this in an orderly way as there are two different sources of randomness.

Firstly, to minimise the randomness due to the partitioning of the data set, it is necessary to have a method that ensures that each data is used for training at least once, and for testing at least once. The most commonly used method is cross-validation. In this method, the data set is split into k disjoint subsets and k experiments are performed. In the k-th experiment, the k subset is separated for testing, and the remaining k-1 substes are used for training, performing a k-fold cross-validation. A common value is k=10, which gives a 10-fold cross-validation. Finally, the test value corresponding to the appropriate metric will be the average value of the values of the k experiments.

A widely used variant of cross-validation is stratified cross-validation. In this case, each subset is created in such a way as to keep the proportion of patterns of each class the same (or similar) as in the original dataset. This is particularly used when the data set is imbalanced.

It is usual to save not only the mean, but also the k values, in order to subsequently perform a paired hypothesis test with another model. To do this, it is necessary that both models have been trained using the same training and test sets.

This way of evaluating the model is often considered to be slightly pessimistic, i.e. the results obtained in tests are slightly worse than those that would be obtained from real training with all available data. In a hold out experiment, as mentioned above, several data are separated for testing. This means that the model is trained with less data than is available, and that by chance the data separated for testing can be of great importance (especially if there is little data). For this reason, when training with less data and possibly no "important" data, hold out is considered a pessimistic assessment. In the same way, cross-validation also separates data for testing, so it does not train on all available data, and is therefore also pessimistic. However, it is guaranteed that all data are used at least once in training and once in testing, thus trying to minimise the impact of chance in separating data, so it is considered only a slightly pessimistic evaluation.

Doing this is as simple as splitting the data set and performing a loop with k iterations in which at the k-th iteration a model is trained and evaluated with the corresponding sets. However, if the model is not deterministic, the result obtained at the k-th iteration will not be meaningful, since it is again dependent on chance. In this case, what needs to be done is a second nested loop within iteration k in which the model is repeatedly trained, and finally an average of the results is made to finally output the result of iteration k. The number of trainings must be high for the average results to be really significant, at least 50 trainings.

### Question

If this second loop is performed with a deterministic model, what will be the standard deviation of the test results obtained? Is there a difference between performing this second loop and averaging the results, or doing a single training?

`Answer here` If that second loop is performed with a deterministic model, the standard deviation of the test results should be 0. If we train the model with the same data and the same initialization each time, you will obtain the same results.
 
The difference between performing the second loop and doing a single training lies in mitigate the non-deterministic effect introduced by the random initialization of the weights. If we were talking about a deterministic model it won't be necesarry.

In this way, it is possible to evaluate a model together with its hyperparameters in solving a problem. A very common situation is to compare several models (or the same model with different hyperparameters), for which this scheme has to be applied with an important caveat: the sets used in the cross-validation must be the same for each model. Since the distribution of patterns in different sets is random, having the same subsets in different runs is achieved by setting the random seed at the beginning of the program to be executed. Setting the random seed not only allows the same subsets to be generated, but is also important in order to be able to repeat the results in different runs.

It is also important to bear in mind that this methodology allows estimating the real performance of a model (although slightly pessimistic). The final model that would be used in production would be the result of training it with all the available patterns, since, as seen in the theory class, and very generally speaking, the more patterns you train with, the better the model will be.

In this assignment, you are asked to:

1. Develop a function called `crossvalidation` that receives a value `N` (equal to the number of patterns), and a value `k` (number of subsets into which the dataset is to be split), and returns a vector of length N, where each element indicates in which subset that pattern should be included.

    To do this function, one possibility is to perform the following steps:
    
    - Create a vetor with k sorted elements, from 1 to k.
    - Create a new vector with repetitions of the previous vector until its length is greater than or equal to N. The functions `repeat` and `ceil` can be used for this purpose.
    - Take the first N values of this vector.
    - Shuffle this vector (using the function `shuffle!` and return it. To use this function, the module `Random` should be loaded.
    
    No loop function should be used in the developed function.

In [1]:
#Corssvalidation function
function crossvalidation(N::Int64, k::Int64)
    #TODO
    # Step 1: Create a vector with k sorted elements
    subsets = collect(1:k)
    println("subsets1",subsets)
    # Step 2: Repeat the vector until its length is greater than or equal to N
    subsets = repeat(subsets, outer = ceil(Int, N / k))
    println("subsets2",subsets)
    # Step 3: Take the first N values of this vector
    subsets = subsets[1:N]
    println("subsets3",subsets)
    # Step 4: Shuffle the vector
    shuffle!(subsets)

    return subsets
end

crossvalidation (generic function with 1 method)

2. Create a new function called `crossvalidation`, which in this case receives as first argument `targets` of type `AbstractArray{Bool,2}` with the desired outputs, and as second argument a value `k` (number of subsets in which the dataset will be split), and returns a vector of length N (equal to the number of rows of targets), where each element indicates in which subset that pattern must be included. This partition has also to be stratified. To do this, the following steps can be followed:

    - Create a vector of indices, with as many values as rows in the `target` matrix.
    - Write a loop that iterates over the classes (columns in the `target` matrix), and does the following:
        - Take the number of elements belonging to that class. This can be done by making a call to the `sum` function applied to the corresponding column.
        - Make a call to the `crossvalidation` function developed earlier passing as parameters this number of elements and the value of k.
        - Update the index vector positions indicated by the corresponding column of the `targets` matrix with the values of the vector resulting from the call to the `crossvalidation` function.
        
        ### Question
        
        Could you perform these 3 operations in a single function call?
        
        ```Answer here```  index_vector = [crossvalidation(sum(targets[:, i]), k) .+ (i - 1) * k for i in 1:size(targets, 2)]
        </br>
    - Return the vector of indices.
    
    As it can be seen in this explanation, a loop iterating all classes can be used in this function. However, you need to make sure that each class has at least k patterns. A usual value is k=10. Therefore, it is important to make sure that you have at least 10 patterns of each class.
        
    ### Question
    
    What would happen if any class has a number of patterns less than k? What would be the consequences for calculating metrics?
    
    ```Answer here``` The metrics would be less reliable and may not accurately represent the performance of the model on a class due to the small sample size.
    
    > If, for whatever reason, it is impossible to ensure that you have at least 10 patterns of each class, one possibility would be to lower the value of k. In this case, consult with the teacher to assess this option, and what impact it might have on the final result of the trained models. In this case, consult with the teacher to assess this option, and what impact it might have on the final result of the trained models.

In [2]:
#Crossvalidation function
function crossvalidation(targets::AbstractArray{Bool,2}, k::Int64)
    #TODO
    N = size(targets, 1)
    indices = collect(1:N)

    for class in 1:size(targets, 2)
        # Step 1: Take the number of elements belonging to that class
        num_elements = sum(targets[:, class])
        println("num_elements",num_elements)
        # Step 2: Call the crossvalidation function
        class_indices = crossvalidation(num_elements, k)
        println("class_indices",class_indices)
        println(size(class_indices))
        # Step 3: Update the index vector positions
        indices[targets[:, class]] = class_indices;
        println("indices",indices)
    end

    shuffle!(indices)  # Shuffle the final index vector

    return indices
end

crossvalidation (generic function with 2 methods)

3. Perform a final function called crossvalidation, but in this case with the first parameter `targets` of type `AbstractArray{<:Any,1}` (i.e. a vector with heterogeneous elements), the same second argument, and perform stratified cross-validation.

    In this case, the steps to follow in this function are not specified. However, they are similar to the previous one. A simple way to do it would be to call the function `oneHotEncoding` passing the vector `targets` as an argument.
    
      ### Question
      
      Could you develop this function without calling oneHotEncoding?
      
      ```Answer here``` It is posible to develop this function without oneHotEncoding, we can work with the heterogeneous targets and adapt the logic within the function to handle them appropriately. (In this case we must adjust how the classes are identified).

In [3]:
# One-hot encoding function
function oneHotEncoding(labels)
    unique_labels = unique(labels)
    encoded_targets = [labels .== label for label in unique_labels]
    return hcat(encoded_targets...)
end
    
#Crossvalidation function for one-hot encoded targets
function crossvalidation(targets::AbstractArray{<:Any, 1}, k)
    println("Onehot")
    encoded_targets = oneHotEncoding(targets)
    return crossvalidation(encoded_targets, k)
end

crossvalidation (generic function with 3 methods)

4. Integrate these functions into the code developed so far and define two functions to train ANNs following the stratified cross-validation strategy. To do this:

- First, it is necessary to set the random seed to ensure that the experiments are repeatable. This can be done with the `seed!` function of the `Random` module.
- Once the data is loaded and encoded, generate an index vector by calling the `crossvalidation` function.
- Create a function called `trainClassANN`, which receives as parameters the topology, the training set and the indices used for cross-validation. Optionally, it can receive the rest of the parameters used in previous assignments. Inside this function, the following steps may be followed:
    - Create a vector with k elements, which will contain the test results of the cross-validation process with the selected metric. If more than one metric is to be used, create one vector per metric.
    - Make a loop with k iterations (k folds) where, within each iteration, 4 matrices are created from the desired input and output matrices by means of the index vector resulting from the previous function. Namely, the desired inputs and outputs for training and test. As always, do this process of creating new matrices without loops.
    - Within this loop, add a call to generate the model with the training set, and test with the corresponding test set according to the value of k. This can be done by calling the `trainClassANN` function developed in previous assignments, passing as parameters the corresponding sets.
    - As indicated in the previous assignment, the training of ANNs is not deterministic, so that, for each iteration k of the cross-validation, it will be necessary to train several ANNs and return the average of the test results (with the selected metric or metrics) in order to have the test value corresponding to this k.
    - Furthermore, in the case of training ANNs, the training set can be split into training and validation if the ratio of patterns to be used for the validation set is greater than 0. To do this, use the `holdOut` function developed in a previous assignment.
    - Once the model has been trained (several times) on each fold, take the result and fill in the vector(s) created earlier (one for each metric).
    - Finally, provide the result of averaging the values of these vectors for each metric together with their standard deviations.
    - As a result of this call, at least the test value in the selected metric(s) should be returned. If the model is not deterministic (as is the case for the ANNs), it will be the average of the results of several trainings.
- Once this function is done, develop a second one, of the same name, so that it accepts as desired outputs a vector instead of an array, as in a previous assignment, and its operation is simply to make a call to this newly developed function.

> **Remarks**:
> - Although we have only seen how to train ANNs, in the next assignment we will use other models contained in another library (Scikit-Learn). The idea is to use the same code used for cross-validation with this global loop, changing only the line in which the model is generated.
> - Note that other Machine Learning models are deterministic, so they do not need the inner loop (whenever they are trained with the same data they return the same outputs), but only the loop for each fold.

In [4]:
using Pkg; Pkg.add("NBInclude")
using NBInclude

# Include the notebooks
@nbinclude("/home/martin/Escritorio/ML1/MIA_ML1/Unit 2 - Multilayer Perceptron_SOLVED.ipynb")
@nbinclude("/home/martin/Escritorio/ML1/MIA_ML1/Unit 3 - Overfitting.ipynb")
@nbinclude("/home/martin/Escritorio/ML1/MIA_ML1/Unit 4.2 - Multiclass classification.ipynb")


    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Matrix{Float32}
Epoch 0: loss: 1.260584
Epoch 1: loss: 1.2415339
Epoch 2: loss: 1.2239745
Epoch 3: loss: 1.2078773
Epoch 4: loss: 1.1932002
Epoch 5: loss: 1.1798925
Epoch 6: loss: 1.1678886
Epoch 7: loss: 1.157115
Epoch 8: loss: 1.1474931
Epoch 9: loss: 1.1389422
Epoch 10: loss: 1.1313783
Epoch 11: loss: 1.1247166
Epoch 12: loss: 1.1188723
Epoch 13: loss: 1.1137608
Epoch 14: loss: 1.1093024
Epoch 15: loss: 1.1054201
Epoch 16: loss: 1.1020427
Epoch 17: loss: 1.0991039
Epoch 18: loss: 1.0965437
Epoch 19: loss: 1.0943077
Epoch 20: loss: 1.092347
Epoch 21: loss: 1.0906181
Epoch 22: loss: 1.0890821
Epoch 23: loss: 1.0877056
Epoch 24: loss: 1.0864574
Epoch 25: loss: 1.0853118
Epoch 26: loss: 1.0842457
Epoch 27: loss: 1.0832385
Epoch 28: loss: 1.0822723
Epoch 29: loss: 1.0813313
Epoch 30: loss: 1.0804017
Epoch 31: loss: 1.0794708
Epoch 32: loss: 1.0785278
Epoch 33: loss: 1.0775626
Epoch 34: loss: 1.0765662
Epoch 35: loss: 1.075531
Epoch 36: loss: 1.0744497
Epoch 37: loss: 1.0733155
Epoch 38: 

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Input vector: [-1.0, -1.0, -0.2]
Output vector (softmax): [0.23665609135556676, 0.23665609135556676, 0.5266878172888664]


confusionMatrix (generic function with 6 methods)

In [50]:
using Random
############################
##First function of train ##
############################

function trainClassANN(topology::AbstractArray{<:Int,1}, 
        trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,2}}, 
        kFoldIndices::Array{Int64,1}; 
        transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)), 
        maxEpochs::Int=1000, minLoss::Real=0.0, learningRate::Real=0.01, repetitionsTraining::Int=1, 
        validationRatio::Real=0.0, maxEpochsVal::Int=20)

    # Check that topology has at least two elements
    @assert length(topology) > 1

    # Check that repetitionsTraining is greater than 0
    @assert repetitionsTraining > 0

    # Check that learning rate is greater than 0
    @assert learningRate > 0
    
    #Check that validationRatio is between 0 and 1
    @assert 0 <= validationRatio <= 1

    # Check that maxEpochs is greater than 0
    @assert maxEpochsVal > 0

    #Check that minLoss is greater than 0
    @assert minLoss >= 0

    # Set random seed for reproducibility
    Random.seed!(1234)
    
    # Get number of folds
    numFolds=maximum(kFoldIndices)

    # Create vector to store test results
    result_confusionMatrix2 = [];
    result_accuracy_2 = [];
    results_error_rate_2 = [];
    bestModel_2=[]
    (inputs, targets) = trainingDataset;
    
    # Check that inputs and targets have the same number of samples
    @assert size(inputs, 1) == size(targets, 1) 

    #Check that trainingDataset and valDataset are not empty
    @assert !isempty(inputs) && !isempty(targets) 
    
    # Loop over k folds
    for k in unique(kFoldIndices)
        # Create training and test sets
        
        trainInputs = inputs[kFoldIndices.!=k,:]
        trainOutputs = targets[kFoldIndices.!=k,:]
        testInputs = inputs[kFoldIndices.==k,:]
        testOutputs = targets[kFoldIndices.==k,:]            

        if validationRatio>0

            testOutputs=BitMatrix(testOutputs')
        else    
            testOutputs=BitMatrix(testOutputs')'

        end
        
        # Train ANN and get test result
        result_confusionMatrix1 = [];
        result_accuracy_1 = [];
        results_error_rate_1 = [];
        bestModel_1=[];
        for i in 1:repetitionsTraining
            
            # Split training set into training and validation sets if validationRatio > 0
            if validationRatio > 0
                train_indices,val_indices,test_indices=holdOut(size(inputs,1),validationRatio,0.2)

                trainInputs, valInputs, testInputs = inputs[train_indices,:], inputs[val_indices,:], inputs[test_indices,:]

                trainOutputs, valOutputs, testOutputs = targets[train_indices,:], targets[val_indices,:], targets[test_indices,:]


            else
                valInputs, valOutputs = reshape([1],1,1),BitMatrix([1,0]')
            end
            
            println(size(trainInputs))
            println(size(trainOutputs))
            println(size(valInputs))
            println(size(valOutputs))
            println(size(testInputs))
            println(size(testOutputs))

            #Check that validationRatio is 0 or trainingDataset and valDataset are not empty
            @assert validationRatio == 0 || (!isempty(valInputs) && !isempty(valOutputs))
            
            model,_, _,_ = trainClassANN(topology, (trainInputs, trainOutputs), 
                transferFunctions=transferFunctions, maxEpochs=maxEpochs, minLoss=minLoss, 
                learningRate=learningRate, validationSet=(valInputs,valOutputs), 
                testSet=(testInputs, testOutputs), maxEpochsVal=maxEpochsVal)
            
                
            outputs=model(trainInputs');
            accuracy1, error_rate, sensitivity, specificity, positive_predictive_value, negative_predictive_value, f_score, confusion_matrix=confusionMatrix(outputs',trainOutputs,weighted=false)
            
            #store results
            push!(bestModel_1, model)
            push!(result_confusionMatrix1, confusion_matrix)
            push!(result_accuracy_1, accuracy1)
            push!(results_error_rate_1, error_rate)

        end

        #Choose the best result
        index=argmax(result_accuracy_1)
        push!(bestModel_2, bestModel_1[index])
        push!(result_confusionMatrix2, result_confusionMatrix1[index])
        push!(result_accuracy_2, result_accuracy_1[index])
        push!(results_error_rate_2, results_error_rate_1[index])

    end
    
    # Calculate mean and standard deviation of test results
    meanAccuracy = mean(result_accuracy_2)
    stdAccuracy = std(result_accuracy_2)

    meanErrorRate = mean(results_error_rate_2)
    stdErrorRate = std(results_error_rate_2)

    meanConfusionMatrix = mean(result_confusionMatrix2)
    stdConfusionMatrix = std(result_confusionMatrix2)

    # Return mean and standard deviation of test results
    return meanAccuracy, stdAccuracy, meanErrorRate, stdErrorRate, meanConfusionMatrix, stdConfusionMatrix
end



trainClassANN (generic function with 4 methods)

In [51]:
############################
##Second function of train #
############################
function trainClassANN(topology::AbstractArray{<:Int,1}, 
                        trainingDataset::Tuple{AbstractArray{<:Real,2}, AbstractArray{Bool,1}}, 
                        kFoldIndices::Array{Int64,1};
                        transferFunctions::AbstractArray{<:Function,1}=fill(σ, length(topology)),
                        maxEpochs::Int=1000, minLoss::Real=0.0, learningRate::Real=0.01,
                        repetitionsTraining::Int=1, validationRatio::Real=0.0, maxEpochsVal::Int=20)
    
    (inputs, targets) = trainingDataset;
    
    # Convert the target vector to a BitMatrix
    targets = BitMatrix(targets')
   
    # Call the original function with the modified target format
    return trainClassANN(topology, (inputs, targets), kFoldIndices;
                        transferFunctions=transferFunctions,
                        maxEpochs=maxEpochs, minLoss=minLoss,
                        learningRate=learningRate, repetitionsTraining=repetitionsTraining,
                        validationRatio=validationRatio, maxEpochsVal=maxEpochsVal)
end


trainClassANN (generic function with 4 methods)

In [54]:
#Test the functions

Random.seed!(42)

using DelimitedFiles;
using Flux;

#Define the parameters
topology = [4,4, 3]; 
learningRate = 0.01;
numMaxEpochs = 1000; 

# Load the dataset
dataset = readdlm("../iris/iris.data",',');
# Prepare the data
inputs = convert(Array{Float32,2}, dataset[:,1:4]);
targets = oneHotEncoding(dataset[:,5]);

# Normalize the inputs
inputs = normalizeMinMax!(inputs);

# Check that the inputs are normalized
@assert(all(minimum(inputs, dims=1) .== 0));
@assert(all(maximum(inputs, dims=1) .== 1));


# Call the crossvalidation function
vector=crossvalidation(targets,10)


# Call the trainClassANN function
meanAccuracy, stdAccuracy, meanErrorRate, stdErrorRate, meanConfusionMatrix, stdConfusionMatrix=trainClassANN(topology, (inputs,targets), vector,repetitionsTraining=10)
println(vector)
# Print the results
println("Accuracy: ", 100*meanAccuracy, " %");
println("Error rate: ", 100*meanErrorRate, " %");
println("Confusion matrix: ", meanConfusionMatrix);
println("Standard deviation of accuracy: ", 100*stdAccuracy, " %");
println("Standard deviation of error rate: ", 100*stdErrorRate, " %");
println("Standard deviation of confusion matrix: ", stdConfusionMatrix);

(135, 4)
(135, 3)
(1, 1)
(1, 2)
(15, 4)
(15, 3)
Epoch 0: Training Loss: 1.3083318 | Validation Loss: 1.3083318 | Test Loss: 1.1905841
Epoch 756: Training Loss: 0.06060055 | Validation Loss: 0.06060055 | Test Loss: 0.01741102
Paramos entrenamiento
Min Loss
(135, 4)
(135, 3)
(1, 1)
(1, 2)
(15, 4)
(15, 3)
Epoch 0: Training Loss: 1.2024398 | Validation Loss: 1.2024398 | Test Loss: 1.1414577
Epoch 709: Training Loss: 0.058927078 | Validation Loss: 0.058927078 | Test Loss: 0.018196046
Paramos entrenamiento
Min Loss
(135, 4)
(135, 3)
(1, 1)
(1, 2)
(15, 4)
(15, 3)
Epoch 0: Training Loss: 1.0968312 | Validation Loss: 1.0968312 | Test Loss: 1.0862504
Epoch 570: Training Loss: 0.055045288 | Validation Loss: 0.055045288 | Test Loss: 0.011888542
Paramos entrenamiento
Min Loss
(135, 4)
(135, 3)
(1, 1)
(1, 2)
(15, 4)
(15, 3)
Epoch 0: Training Loss: 1.2460363 | Validation Loss: 1.2460363 | Test Loss: 1.2460424
Epoch 765: Training Loss: 0.056679077 | Validation Loss: 0.056679077 | Test Loss: 0.01568369